In [288]:
# Imports
import sklearn
import scipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [289]:
df = pd.read_csv('AviationData2.csv')

In [290]:
df

,NtsbNo,EventType,Mkey,EventDate,City,State,Country,ReportNo,N,HasSafetyRec,...,PurposeOfFlight,FAR,AirCraftDamage,WeatherCondition,Operator,ReportStatus,RepGenFlag,DocketUrl,DocketPublishDate,Unnamed: 37
0,DCA24WA087,ACC,193738,2024-02-01T03:08:00Z,Tokoname,NaN,Japan,NaN,JA899A,False,...,NaN,NUSC,NaN,NaN,All Nippon Airways,NaN,False,NaN,NaN,NaN
1,GAA24WA073,ACC,193695,2024-01-20T12:30:00Z,Murcott,NaN,United Kingdom,NaN,GNEWT,False,...,NaN,NUSN,Unknown,NaN,NaN,NaN,False,NaN,NaN,NaN
2,GAA24WA067,ACC,193623,2024-01-07T23:07:00Z,Lizard Island,NaN,Australia,NaN,VH-NWJ,False,...,NaN,UNK,Substantial,NaN,NaN,NaN,False,NaN,NaN,NaN
3,DCA24WA071,ACC,193655,2024-01-06T03:28:00Z,Mar del Plata,NaN,Argentina,NaN,LV-KJE,False,...,NaN,NUSC,Substantial,NaN,NaN,NaN,False,NaN,NaN,NaN
4,CEN24LA075,ACC,193582,2024-01-01T16:40:00Z,Mansfield,Louisiana,United States,NaN,N9729L,False,...,PERS,091,Destroyed,VMC,NaN,Completed,False,https://data.ntsb.gov/Docket?ProjectID=193582,2/8/2024 6:00:00 PM,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142461,NYC62A0005,ACC,115930,1962-01-16T05:00:00Z,LIMESTONE,Maine,United States,NaN,N9903F,False,...,NaN,091,Substantial,VFR,NaN,Completed,False,NaN,NaN,NaN
142462,MIA62A0003,ACC,123569,1962-01-14T05:00:00Z,BIRMINGHAM,Alabama,United States,NaN,N5540,False,...,NaN,NaN,Substantial,VFR,EASTERN AIR LINES INC,Completed,False,NaN,NaN,NaN
142463,LAX62A0002,ACC,150606,1962-01-13T05:00:00Z,FILLMORE,California,United States,NaN,N37538,False,...,NaN,NaN,NaN,VFR,UNITED AIR LINES INC,Completed,False,NaN,NaN,NaN
142464,MIA62A0001,ACC,139827,1962-01-01T05:00:00Z,CHARLOTTE,North Carolina,United States,NaN,N2703R,False,...,NaN,NaN,Substantial,VFR,PIEDMONT AVIATION INC,Completed,False,NaN,NaN,NaN


In [291]:
df.shape

(142466, 38)

## Data cleaning and preprocessing

In [292]:
df.isna().sum().sort_values(ascending=False)

Unnamed: 37            142466
ReportNo               142155
Scheduled              134676
DocketUrl              126575
DocketPublishDate      126575
Operator               105813
AirportID              102797
ProbableCause           97167
HighestInjuryLevel      86531
EventID                 65955
AirportName             46406
OriginalPublishDate     13508
PurposeOfFlight          8290
State                    5358
FAR                      4374
ReportType               3488
ReportStatus             3416
NumberOfEngines          2621
WeatherCondition         2128
AirCraftDamage           1485
Country                   310
Model                      36
City                       31
N                          28
Make                        5
RepGenFlag                  0
NtsbNo                      0
AmateurBuilt                0
AirCraftCategory            0
EventType                   0
Latitude                    0
MinorInjuryCount            0
SeriousInjuryCount          0
FatalInjur

In [293]:
df.drop_duplicates(inplace=True)

In [294]:
df.shape

(142466, 38)

In [295]:
# Let's remove columns we are not interested in
df.columns

Index(['NtsbNo', 'EventType', 'Mkey', 'EventDate', 'City', 'State', 'Country',
       'ReportNo', 'N', 'HasSafetyRec', 'ReportType', 'OriginalPublishDate',
       'HighestInjuryLevel', 'FatalInjuryCount', 'SeriousInjuryCount',
       'MinorInjuryCount', 'ProbableCause', 'EventID', 'Latitude', 'Longitude',
       'Make', 'Model', 'AirCraftCategory', 'AirportID', 'AirportName',
       'AmateurBuilt', 'NumberOfEngines', 'Scheduled', 'PurposeOfFlight',
       'FAR', 'AirCraftDamage', 'WeatherCondition', 'Operator', 'ReportStatus',
       'RepGenFlag', 'DocketUrl', 'DocketPublishDate', 'Unnamed: 37'],
      dtype='object')

In [296]:
df.drop(columns=[
    'Mkey',
    'ReportNo',
    'N', # N is the ID of the plane
    'HasSafetyRec',
    'ReportType',
    'EventID',
    'Latitude',
    'Longitude',
    'Unnamed: 37',
    'DocketPublishDate',
    'DocketUrl',
    'AirportID',
    'AirportName',
    'ReportStatus',
    'Operator',
    'AmateurBuilt',
    'NtsbNo',
    'OriginalPublishDate',
    'FAR',
    'RepGenFlag',
    'Scheduled',
    'EventType'
    
], inplace=True)

In [297]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142466 entries, 0 to 142465
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   EventDate           142466 non-null  object
 1   City                142435 non-null  object
 2   State               137108 non-null  object
 3   Country             142156 non-null  object
 4   HighestInjuryLevel  55935 non-null   object
 5   FatalInjuryCount    142466 non-null  int64 
 6   SeriousInjuryCount  142466 non-null  int64 
 7   MinorInjuryCount    142466 non-null  int64 
 8   ProbableCause       45299 non-null   object
 9   Make                142461 non-null  object
 10  Model               142430 non-null  object
 11  AirCraftCategory    142466 non-null  object
 12  NumberOfEngines     139845 non-null  object
 13  PurposeOfFlight     134176 non-null  object
 14  AirCraftDamage      140981 non-null  object
 15  WeatherCondition    140338 non-null  object
dtypes:

In [298]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142466 entries, 0 to 142465
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   EventDate           142466 non-null  object
 1   City                142435 non-null  object
 2   State               137108 non-null  object
 3   Country             142156 non-null  object
 4   HighestInjuryLevel  55935 non-null   object
 5   FatalInjuryCount    142466 non-null  int64 
 6   SeriousInjuryCount  142466 non-null  int64 
 7   MinorInjuryCount    142466 non-null  int64 
 8   ProbableCause       45299 non-null   object
 9   Make                142461 non-null  object
 10  Model               142430 non-null  object
 11  AirCraftCategory    142466 non-null  object
 12  NumberOfEngines     139845 non-null  object
 13  PurposeOfFlight     134176 non-null  object
 14  AirCraftDamage      140981 non-null  object
 15  WeatherCondition    140338 non-null  object
dtypes:

In [299]:
df.isna().sum().sort_values(ascending=False)

ProbableCause         97167
HighestInjuryLevel    86531
PurposeOfFlight        8290
State                  5358
NumberOfEngines        2621
WeatherCondition       2128
AirCraftDamage         1485
Country                 310
Model                    36
City                     31
Make                      5
EventDate                 0
FatalInjuryCount          0
SeriousInjuryCount        0
MinorInjuryCount          0
AirCraftCategory          0
dtype: int64

In [300]:
# Probable cause would be a valuable metric, however that information is missing from a really large number of entries. Therefore, the column will be dropped.
df.drop('ProbableCause', axis=1, inplace=True)  

In [301]:
# State is not super important, so we will drop it as well, along with PurposeOfFlight, Make and Model
df.drop(columns=['PurposeOfFlight', 'State', 'Make', 'Model'], axis=1, inplace=True)  

In [302]:
df.isna().sum().sort_values(ascending=False)

HighestInjuryLevel    86531
NumberOfEngines        2621
WeatherCondition       2128
AirCraftDamage         1485
Country                 310
City                     31
EventDate                 0
FatalInjuryCount          0
SeriousInjuryCount        0
MinorInjuryCount          0
AirCraftCategory          0
dtype: int64

In [303]:
# HighestInjuryLevel can be extracted from the injury counts, which don't have any missing values, so we will remove this column as well and extract the information later
df.drop('HighestInjuryLevel', axis=1, inplace=True)  

In [304]:
df.isna().sum().sort_values(ascending=False)

NumberOfEngines       2621
WeatherCondition      2128
AirCraftDamage        1485
Country                310
City                    31
EventDate                0
FatalInjuryCount         0
SeriousInjuryCount       0
MinorInjuryCount         0
AirCraftCategory         0
dtype: int64

In [305]:
# Now we are left with all the information we want and reduced the number of missing values significantly
df.shape

(142466, 10)

In [306]:
# Given the shear amount of data, it's acceptable to simply drop the null values
df.dropna(inplace=True)
df.shape

(137512, 10)

In [307]:
df.isna().sum().sort_values(ascending=False)

EventDate             0
City                  0
Country               0
FatalInjuryCount      0
SeriousInjuryCount    0
MinorInjuryCount      0
AirCraftCategory      0
NumberOfEngines       0
AirCraftDamage        0
WeatherCondition      0
dtype: int64

In [308]:
# Now let's change the types of the numerical data that are mistakenly described as objects
df.EventDate = pd.to_datetime(df.EventDate)

In [309]:
df.NumberOfEngines.value_counts()

NumberOfEngines
1        117795
2         16947
1,1        1342
4           578
3           221
2,2         132
2,1         132
1,2         122
0           108
4,2          19
1,           18
,            12
,1           11
2,3          10
1,0          10
0,1           8
4,4           6
2,4           5
1,4           4
2,            4
1,3           3
3,4           3
3,2           3
4,1           3
3,1           3
,2            3
3,3           1
2,2,2         1
4,3           1
1,1,1         1
0,0           1
2,0           1
1,1,0         1
4,            1
,4            1
6,1           1
Name: count, dtype: int64

In [310]:
# The number of engines seems to be in a strange format, let's check
df.loc[df.NumberOfEngines == '1,1']

,EventDate,City,Country,FatalInjuryCount,SeriousInjuryCount,MinorInjuryCount,AirCraftCategory,NumberOfEngines,AirCraftDamage,WeatherCondition
145,2023-08-01 17:37:00+00:00,Long Beach,United States,0,0,0,"AIR,AIR","1,1","Minor,Substantial",VMC
314,2023-05-22 11:00:00+00:00,Gueydan,United States,0,0,0,"AIR,AIR","1,1","Substantial,Substantial",VMC
437,2023-03-17 13:40:00+00:00,Mesa,United States,0,1,0,"AIR,AIR","1,1","Substantial,Substantial",VMC
450,2023-03-07 14:04:00+00:00,Hillsboro,United States,0,0,0,"AIR,AIR","1,1","Substantial,Minor",VMC
561,2022-12-27 05:00:00+00:00,Brookshire,United States,0,0,0,"AIR,AIR","1,1","Substantial,Minor",VMC
...,...,...,...,...,...,...,...,...,...,...
140972,1964-04-30 04:00:00+00:00,ENID,United States,0,0,0,"AIR,AIR","1,1","Minor,Substantial",VFR
141161,1964-04-15 04:00:00+00:00,TOLLESON,United States,0,0,0,"AIR,AIR","1,1","Minor,Minor",VFR
141779,1964-02-22 05:00:00+00:00,ALBUQUERQUE,United States,0,0,0,"AIR,AIR","1,1","Substantial,Minor",VFR
142110,1964-01-22 05:00:00+00:00,MOULTRIE,United States,0,0,0,"AIR,AIR","1,1","Substantial,Minor",VFR


In [311]:
# We can see now that more than one aircraft has been involved in this accident, so the number for each airplane is separated by a comma. Let's check for aircraft type
df.AirCraftCategory.value_counts()

AirCraftCategory
AIR            135649
AIR,AIR          1762
AIR,HELI           32
HELI,AIR           24
AIR,GLI            21
GLI,AIR            13
AIR,ULTR            4
AIR,AIR,AIR         2
AIR,UNK             1
AIR,AIR,GLI         1
PPAR,AIR            1
AIR,                1
GYRO,AIR            1
Name: count, dtype: int64

In [312]:
# We are only interested in accidents involving airplanes, so we can remove some of this data
itemList = ['HELI', 'UNK', 'GLI', 'ULTR', 'GYRO', 'PPAR']
for string in itemList:
    df.drop(df[df.AirCraftCategory.str.contains(string)].index, inplace=True)

In [313]:
df.AirCraftCategory.value_counts()

AirCraftCategory
AIR            135649
AIR,AIR          1762
AIR,AIR,AIR         2
AIR,                1
Name: count, dtype: int64

In [314]:
df.drop(df[df.AirCraftCategory == 'AIR,'].index, inplace=True)

In [315]:
df.AirCraftCategory.value_counts()

AirCraftCategory
AIR            135649
AIR,AIR          1762
AIR,AIR,AIR         2
Name: count, dtype: int64

## Exploratory analysis and attribute engineering

In [316]:
# Let's extract the date and the time of the accidents
df.EventDate = pd.to_datetime(df.EventDate)
df.EventDate

4        2024-01-01 16:40:00+00:00
5        2023-12-28 11:57:00+00:00
6        2023-12-26 13:15:00+00:00
9        2023-12-12 17:44:00+00:00
10       2023-12-11 13:15:00+00:00
                    ...           
142460   1962-01-20 05:00:00+00:00
142461   1962-01-16 05:00:00+00:00
142462   1962-01-14 05:00:00+00:00
142464   1962-01-01 05:00:00+00:00
142465   1948-10-24 16:55:00+00:00
Name: EventDate, Length: 137413, dtype: datetime64[ns, UTC]

In [317]:
# Let's first extract some relevant information from the data and create new columns
df['EventTime'] = pd.to_datetime(df.EventDate.dt.strftime('%H:%M'))

In [318]:
df.EventTime

4        2024-02-14 16:40:00
5        2024-02-14 11:57:00
6        2024-02-14 13:15:00
9        2024-02-14 17:44:00
10       2024-02-14 13:15:00
                 ...        
142460   2024-02-14 05:00:00
142461   2024-02-14 05:00:00
142462   2024-02-14 05:00:00
142464   2024-02-14 05:00:00
142465   2024-02-14 16:55:00
Name: EventTime, Length: 137413, dtype: datetime64[ns]

In [319]:
df.EventTime = df.EventTime.dt.time

In [320]:
df.EventTime = df.EventTime.astype(str).str.replace(':', '')


In [321]:
df.EventTime

4         164000
5         115700
6         131500
9         174400
10        131500
           ...  
142460    050000
142461    050000
142462    050000
142464    050000
142465    165500
Name: EventTime, Length: 137413, dtype: object

In [322]:
df.EventTime = df.EventTime.astype(int)
df.EventTime

4         164000
5         115700
6         131500
9         174400
10        131500
           ...  
142460     50000
142461     50000
142462     50000
142464     50000
142465    165500
Name: EventTime, Length: 137413, dtype: int32

In [323]:
df.columns

Index(['EventDate', 'City', 'Country', 'FatalInjuryCount',
       'SeriousInjuryCount', 'MinorInjuryCount', 'AirCraftCategory',
       'NumberOfEngines', 'AirCraftDamage', 'WeatherCondition', 'EventTime'],
      dtype='object')

In [324]:
df.EventDate = pd.to_datetime(df.EventDate.dt.strftime('%Y-%m-%d'))

In [325]:
df.EventDate

4        2024-01-01
5        2023-12-28
6        2023-12-26
9        2023-12-12
10       2023-12-11
            ...    
142460   1962-01-20
142461   1962-01-16
142462   1962-01-14
142464   1962-01-01
142465   1948-10-24
Name: EventDate, Length: 137413, dtype: datetime64[ns]

In [326]:
df.head()

,EventDate,City,Country,FatalInjuryCount,SeriousInjuryCount,MinorInjuryCount,AirCraftCategory,NumberOfEngines,AirCraftDamage,WeatherCondition,EventTime
4,2024-01-01,Mansfield,United States,0,0,0,AIR,1,Destroyed,VMC,164000
5,2023-12-28,Lake Havasu,United States,0,0,1,AIR,1,Substantial,VMC,115700
6,2023-12-26,Ponca City,United States,0,0,0,AIR,2,Substantial,VMC,131500
9,2023-12-12,Sidney,United States,0,0,0,AIR,1,Substantial,VMC,174400
10,2023-12-11,Newport News,United States,0,0,0,AIR,1,Unknown,VMC,131500


In [327]:
df.set_index(df.EventDate, inplace=True)
df.drop(columns='EventDate', inplace=True)
df.head()

,City,Country,FatalInjuryCount,SeriousInjuryCount,MinorInjuryCount,AirCraftCategory,NumberOfEngines,AirCraftDamage,WeatherCondition,EventTime
EventDate,,,,,,,,,,
2024-01-01,Mansfield,United States,0,0,0,AIR,1,Destroyed,VMC,164000
2023-12-28,Lake Havasu,United States,0,0,1,AIR,1,Substantial,VMC,115700
2023-12-26,Ponca City,United States,0,0,0,AIR,2,Substantial,VMC,131500
2023-12-12,Sidney,United States,0,0,0,AIR,1,Substantial,VMC,174400
2023-12-11,Newport News,United States,0,0,0,AIR,1,Unknown,VMC,131500


In [328]:
# Let's extract the year from the date
df['Year'] = df.index.year

In [329]:
df.head()

,City,Country,FatalInjuryCount,SeriousInjuryCount,MinorInjuryCount,AirCraftCategory,NumberOfEngines,AirCraftDamage,WeatherCondition,EventTime,Year
EventDate,,,,,,,,,,,
2024-01-01,Mansfield,United States,0,0,0,AIR,1,Destroyed,VMC,164000,2024
2023-12-28,Lake Havasu,United States,0,0,1,AIR,1,Substantial,VMC,115700,2023
2023-12-26,Ponca City,United States,0,0,0,AIR,2,Substantial,VMC,131500,2023
2023-12-12,Sidney,United States,0,0,0,AIR,1,Substantial,VMC,174400,2023
2023-12-11,Newport News,United States,0,0,0,AIR,1,Unknown,VMC,131500,2023


In [330]:
# Let's create a new column to store the accident severity based on the amount and type of injury
df['AccidentSeverity'] = pd.Series()

In [331]:
df.loc[df.FatalInjuryCount > 0, 'AccidentSeverity'] = 'Fatal'
df.loc[(df.SeriousInjuryCount > 0) & (df.FatalInjuryCount == 0), 'AccidentSeverity'] = 'Serious'
df.loc[(df.MinorInjuryCount > 0) & (df.SeriousInjuryCount == 0) & (df.FatalInjuryCount == 0), 'AccidentSeverity'] = 'Minor'
df.loc[(df.MinorInjuryCount == 0) & (df.SeriousInjuryCount == 0) & (df.FatalInjuryCount == 0), 'AccidentSeverity'] = 'No injuries'

In [332]:
# Let's rename these columns for brevity
df.rename(columns={'FatalInjuryCount': 'FatalInjuries', 
                   'SeriousInjuryCount': 'SeriousInjuries',
                   'MinorInjuryCount': 'MinorInjuries'}, inplace=True)

In [333]:
df.AccidentSeverity.value_counts()

AccidentSeverity
No injuries    85000
Fatal          22089
Minor          18800
Serious        11524
Name: count, dtype: int64

In [334]:
df.AirCraftCategory.value_counts()

AirCraftCategory
AIR            135649
AIR,AIR          1762
AIR,AIR,AIR         2
Name: count, dtype: int64

In [335]:
df['NumberOfAircraftInvolved'] = df.AirCraftCategory.str.count('AIR')

In [336]:
df.NumberOfAircraftInvolved.value_counts()

NumberOfAircraftInvolved
1    135649
2      1762
3         2
Name: count, dtype: int64

In [337]:
df.head()

,City,Country,FatalInjuries,SeriousInjuries,MinorInjuries,AirCraftCategory,NumberOfEngines,AirCraftDamage,WeatherCondition,EventTime,Year,AccidentSeverity,NumberOfAircraftInvolved
EventDate,,,,,,,,,,,,,
2024-01-01,Mansfield,United States,0,0,0,AIR,1,Destroyed,VMC,164000,2024,No injuries,1
2023-12-28,Lake Havasu,United States,0,0,1,AIR,1,Substantial,VMC,115700,2023,Minor,1
2023-12-26,Ponca City,United States,0,0,0,AIR,2,Substantial,VMC,131500,2023,No injuries,1
2023-12-12,Sidney,United States,0,0,0,AIR,1,Substantial,VMC,174400,2023,No injuries,1
2023-12-11,Newport News,United States,0,0,0,AIR,1,Unknown,VMC,131500,2023,No injuries,1


In [338]:
# Now we can remove the category, since we already have the number of airplanes involved and all aircraft are airplanes
df.drop(columns=['AirCraftCategory'], inplace=True)
df.head()

,City,Country,FatalInjuries,SeriousInjuries,MinorInjuries,NumberOfEngines,AirCraftDamage,WeatherCondition,EventTime,Year,AccidentSeverity,NumberOfAircraftInvolved
EventDate,,,,,,,,,,,,
2024-01-01,Mansfield,United States,0,0,0,1,Destroyed,VMC,164000,2024,No injuries,1
2023-12-28,Lake Havasu,United States,0,0,1,1,Substantial,VMC,115700,2023,Minor,1
2023-12-26,Ponca City,United States,0,0,0,2,Substantial,VMC,131500,2023,No injuries,1
2023-12-12,Sidney,United States,0,0,0,1,Substantial,VMC,174400,2023,No injuries,1
2023-12-11,Newport News,United States,0,0,0,1,Unknown,VMC,131500,2023,No injuries,1


In [339]:
df.WeatherCondition.value_counts()

WeatherCondition
VFR        69498
VMC        56040
IFR         5523
IMC         4924
UNK          864
Unknown      564
Name: count, dtype: int64

In [340]:
df.loc[df.WeatherCondition == 'Unknown', 'WeatherCondition'] = 'UNK'

In [341]:
df.WeatherCondition.value_counts()

WeatherCondition
VFR    69498
VMC    56040
IFR     5523
IMC     4924
UNK     1428
Name: count, dtype: int64

In [342]:
df.head()

,City,Country,FatalInjuries,SeriousInjuries,MinorInjuries,NumberOfEngines,AirCraftDamage,WeatherCondition,EventTime,Year,AccidentSeverity,NumberOfAircraftInvolved
EventDate,,,,,,,,,,,,
2024-01-01,Mansfield,United States,0,0,0,1,Destroyed,VMC,164000,2024,No injuries,1
2023-12-28,Lake Havasu,United States,0,0,1,1,Substantial,VMC,115700,2023,Minor,1
2023-12-26,Ponca City,United States,0,0,0,2,Substantial,VMC,131500,2023,No injuries,1
2023-12-12,Sidney,United States,0,0,0,1,Substantial,VMC,174400,2023,No injuries,1
2023-12-11,Newport News,United States,0,0,0,1,Unknown,VMC,131500,2023,No injuries,1


In [343]:
df.Country.value_counts()

Country
United States                     135946
Mexico                               313
Bahamas                              263
Canada                               170
Brazil                                44
                                   ...  
French Guiana                          1
St Vincent And The Grenadines          1
Republic Of Korea                      1
Federated States Of Micronesia         1
Wake Island                            1
Name: count, Length: 118, dtype: int64

In [344]:
df.AirCraftDamage.value_counts()

AirCraftDamage
Substantial                          105543
Destroyed                             29682
Substantial,Substantial                 618
Minor                                   398
Destroyed,Destroyed                     331
Minor,Substantial                       229
Substantial,Minor                       222
Destroyed,Substantial                   132
Substantial,Destroyed                   121
Destroyed,Minor                          35
Minor,Destroyed                          30
Unknown                                  26
Substantial,None                         20
None,Substantial                         14
None,Minor                                2
None,Destroyed                            2
Minor,Minor                               2
Substantial,Unknown                       1
Destroyed,Substantial,Substantial         1
Destroyed,None                            1
Substantial,Substantial,Minor             1
,Destroyed                                1
Minor,None       

In [350]:
# To help our model, let's use the most severe damage to give this column only one numeric value that represents the amount of damage. (We could also use the average, but let's do it like proposed for simplicity)
df.loc[df.AirCraftDamage.str.contains('Destroyed'), 'AirCraftDamage'] = '3'
df.loc[df.AirCraftDamage.str.contains('Substantial'), 'AirCraftDamage'] = '2'
df.loc[df.AirCraftDamage.str.contains('Minor'), 'AirCraftDamage'] = '1'
df.loc[df.AirCraftDamage.str.contains('Unknown'), 'AirCraftDamage'] = '0'

# Now let's transform them to numbers
df.AirCraftDamage = df.AirCraftDamage.astype(int)
df.AirCraftDamage

EventDate
2024-01-01    3
2023-12-28    2
2023-12-26    2
2023-12-12    2
2023-12-11    0
             ..
1962-01-20    2
1962-01-16    2
1962-01-14    2
1962-01-01    2
1948-10-24    3
Name: AirCraftDamage, Length: 137413, dtype: int32

In [351]:
df.describe()

,FatalInjuries,SeriousInjuries,MinorInjuries,AirCraftDamage,EventTime,Year,NumberOfAircraftInvolved
count,137413.000000,137413.000000,137413.000000,137413.000000,137413.000000,137413.000000,137413.000000
mean,0.380211,0.165559,0.277150,2.217454,87978.830242,1983.454549,1.012852
std,3.003200,0.809989,1.237051,0.420915,57440.550840,15.658764,0.112764
min,0.000000,0.000000,0.000000,0.000000,0.000000,1948.000000,1.000000
25%,0.000000,0.000000,0.000000,2.000000,40000.000000,1970.000000,1.000000
50%,0.000000,0.000000,0.000000,2.000000,50000.000000,1979.000000,1.000000
75%,0.000000,0.000000,0.000000,2.000000,140000.000000,1994.000000,1.000000
max,574.000000,86.000000,137.000000,3.000000,235900.000000,2024.000000,3.000000


## Model Training